In [ ]:
! pip install gekko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 25.6 MB/s eta 0:00:00


In [ ]:
from gekko import GEKKO
import numpy as np

initial_capacity = 4 #MWh initial capacity of battery
max_capacity = 4 #MWh
capacity_per_cycle = 1.0 #MWh
speed = 1 #MW
max_cycle = 3
cycle_cost = 50 #€

num_of_prices = 96

np.random.seed(0)
prices= np.random.randint(low=-100,high=100, size=num_of_prices).reshape(-1,1)
# prices = np.array([[-100],[100],[100],[-100]])
m = GEKKO()
m.options.SOLVER=1  # APOPT is an MINLP solver

m.solver_options = ['minlp_maximum_iterations 1000']



X = m.Array(m.Var,num_of_prices, integer=True)
c = m.Array(m.Param,num_of_prices)
# define optimization variables
for i in range(num_of_prices):
        X[i].lower = -1
        X[i].upper = 1

# define prices
for i in range(num_of_prices):
    c[i].value = prices[i]

# add capacity constraint
_sum = initial_capacity
for i in range(num_of_prices):
    _sum += - capacity_per_cycle/4 * (X[i])
    # limit max and min capacity
    m.Equation(_sum <= max_capacity)
    m.Equation(_sum >= 0)

# # add cycle constraint
_sum = 0
for i in range(num_of_prices):
    _sum += 0.5 * X[i] * X[i] * capacity_per_cycle/4
m.Equation(_sum <= max_cycle*max_capacity)

# define objective
profit = 0
for i in range(num_of_prices):
    profit += X[i] * c[i]                       # arbitrage profit
    profit += - cycle_cost * X[i] * X[i] * 0.5  # add cycle cost in objective

# Define cost
m.Obj(-profit) # maximize profit (multiply by -1 to minimize)

# Solve
m.solve(disp=True)
print("prices:", prices.T)
print("X:", X)
print("Buy: -1, Hold: 0, Sell: 1")
print("Profit:", - m.options.objfcnval)

apm 34.74.171.169_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          385
   Intermediates:            0
   Connections  :            0
   Equations    :          194
   Residuals    :          194
 
 Number of state variables:            289
 Number of total equations: -          193
 Number of slack variables: -          193
 ---------------------------------------
 Degrees of freedom       :            -97
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.28 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -2.73E+03 Gap:       NaN
--Integer Solutio

In [ ]:
np.hstack((X.reshape(-1,1), prices.reshape(-1,1)))

array([[[1.0], 72],
       [[-1.0], -53],
       [[1.0], 17],
       [[1.0], 92],
       [[0.0], -33],
       [[1.0], 95],
       [[0.0], 3],
       [[-1.0], -91],
       [[-1.0], -79],
       [[-1.0], -64],
       [[0.0], -13],
       [[0.0], -30],
       [[0.0], -12],
       [[1.0], 40],
       [[-1.0], -42],
       [[1.0], 93],
       [[-1.0], -61],
       [[0.0], -13],
       [[1.0], 74],
       [[0.0], -12],
       [[0.0], -19],
       [[1.0], 65],
       [[-1.0], -75],
       [[0.0], -23],
       [[0.0], -28],
       [[-1.0], -91],
       [[1.0], 48],
       [[0.0], 15],
       [[1.0], 97],
       [[0.0], -21],
       [[1.0], 75],
       [[1.0], 92],
       [[0.0], -18],
       [[0.0], -1],
       [[1.0], 77],
       [[-1.0], -71],
       [[1.0], 47],
       [[1.0], 47],
       [[1.0], 42],
       [[1.0], 67],
       [[-1.0], -68],
       [[1.0], 93],
       [[-1.0], -91],
       [[1.0], 85],
       [[1.0], 27],
       [[-1.0], -68],
       [[-1.0], -69],
       [[1.0], 51],
    